### Installation

After the installation is done, make sure to restart the session on Google Colab

In [1]:
!pip install openagi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.1/816.1 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 24.2 MB/

In [2]:
!pip install langchain_groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.52
    Uninstalling langchain-core-0.1.52:
      Successfully uninstalled langchain-core-0.1.52
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.8 requires langchain-core<0.2,>=0.1.24, but you have langchain-core 0.2.37 which is incompatible.
langchain-community 0.0.21 requires langchain-core<0.2,>=0.1.24, but you have langchain-core 0.2.37 which is incompatible.
langchain-experimental 0.0.53 requires langchain-core<0.2.0,>=0.1.27, but you have langchain-core 0.2.37 which is incompatible.
langchain-openai 0.0.6 requires langchain-core<0.2,>=0.1.16, but you have langchain-core 0.2.37 which is incompati

In [3]:
!pip install yt-dlp youtube-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.1/170.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 59.1 MB/s eta 0:00:00


In [4]:
from openagi.actions.tools.ddg_search import DuckDuckGoSearch
from openagi.agent import Admin
from openagi.llms.groq import GroqModel
from openagi.memory import Memory
from openagi.planner.task_decomposer import TaskPlanner
from openagi.agent import Worker

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Get your OpenAI API key

### Configure the Large Language Model

Documentation of another supported LLMs: [openagi.aiplanet.com/llms](https://openagi.aiplanet.com/components/llm)

Note: Its recommended to have OpenAI or Azure OpenAI for better quality

In [55]:
import os
from getpass import getpass

os.environ['GROQ_API_KEY'] = getpass("Enter your Groq key:")
os.environ['GROQ_MODEL'] = "llama-3.1-70b-versatile"
os.environ['GROQ_TEMP'] = "0.1"

Enter your Groq key:··········


In [56]:
config = GroqModel.load_from_env_config()
llm = GroqModel(config=config)

### Multi-Agent Architecture

**Define Workers**

Workers are special type of classes, responsible for carrying out the tasks assigned by the class "Admin". They utilize tools such as internet search engines, LLMs, and document writers to perform their tasks. Additionally, they can determine which tools to use from a predefined set.

In [57]:
from openagi.actions.files import WriteFileAction, ReadFileAction
from openagi.actions.tools.webloader import WebBaseContextTool

Similarly to how a large task like writing a blog is decomposed into smaller steps such as researching, drafting, and publishing, the admin can define a large task and split it into smaller tasks that are then assigned to the workers.


The primary components, ``TaskWorker``, provide a structured way to define and execute tasks. ``The TaskWorker`` class specializes in executing specific tasks assigned by the planner.

In [58]:
feedback_collector = Worker(
        role="Customer Feedback Collector",
        instructions="Gather customer feedback specifically about Zuku from various online platforms, including social media, review sites, and forums. Focus on identifying common themes and sentiments related to this product.",
        actions=[DuckDuckGoSearch, WebBaseContextTool, WriteFileAction],
    )

data_analyst = Worker(
    role="Data Analyst",
    instructions="Analyze the collected customer feedback data related to Zuku to identify key trends, recurring issues, and overall customer sentiment. Use statistical tools to quantify the data and provide actionable insights.",
    actions=[ReadFileAction, DuckDuckGoSearch, WebBaseContextTool, WriteFileAction],
)

report_creator = Worker(
    role="Report Creator",
    instructions="Develop a comprehensive customer feedback analysis report based on the data analysis for Zuku. Highlight key findings, trends, and recommendations for improving the product. Ensure the report is well-structured and visually appealing.",
    actions=[ReadFileAction, DuckDuckGoSearch, WebBaseContextTool, WriteFileAction],
)

### Define Admin Agent

Imagine Admin as the master task executor who is responsible for all the major configurations for the execution. From planning of tasks to execution, and defining the brain which is what LLM to use and whether or not to use memory.

Admin is the decision-maker that understand the specifications of tasks and execute them in a more human-like manner.

In [59]:
admin = Admin(
        actions=[DuckDuckGoSearch],
        planner=TaskPlanner(human_intervene=False),
        memory=Memory(),
        llm=llm,
    )
admin.assign_workers([feedback_collector, data_analyst, report_creator])

## Run the Admin

Check your directory, all the Workers tasks results is saved inside ``Log files``.

Once the execution is completed, it will save the results of the Report Creator Agent inside the txt file.

In [61]:
res = admin.run(
        query="Create a customer feedback analysis report for Zuku.",
        description="Collect and analyze customer feedback specifically for Zuku from multiple online sources. Identify common themes, recurring issues, and overall customer sentiment. Develop a comprehensive report that provides actionable insights and recommendations for improving Zuku. Ensure the report is detailed, well-organized, and visually appealing.",
    )



Tasks: [{'task_name': 'Search for customer feedback on Zuku from various online platforms', 'description': 'Use DuckDuckGo to search for customer feedback on Zuku from various online platforms, including social media, review sites, and forums.', 'worker_id': '94f827c826df4727ab457ac70ac71650'}, {'task_name': 'Extract context from search results', 'description': 'Extract actual context from the search results using WebBaseContextTool to gather relevant information about Zuku.', 'worker_id': '94f827c826df4727ab457ac70ac71650'}, {'task_name': 'Summarize extracted context', 'description': 'Summarize the extracted context using WebBaseContextTool to condense the information into a concise format.', 'worker_id': '94f827c826df4727ab457ac70ac71650'}, {'task_name': 'Read summarized context', 'description': 'Read the summarized context to prepare for data analysis.', 'worker_id': '2257733837e84622904d8c7d479d8380'}, {'task_name': 'Analyze customer feedback data', 'description': 'Analyze the co

ERROR:root:Error running action: 1 validation error for ReadFileAction
file_path
  Field required [type=missing, input_value={'description': 'Read the...cretStr('**********')))}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
ERROR:root:Error running action: [Errno 2] No such file or directory: '/content/summarized_context.txt'
ERROR:root:Error running action: 1 validation error for ReadFileAction
file_path
  Field required [type=missing, input_value={'description': 'Read sum...cretStr('**********')))}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
ERROR:root:Error running action: [Errno 2] No such file or directory: '/content/summarized_customer_feedback.txt'
ERROR:root:Error running action: 1 validation error for ReadFileAction
file_path
  Field required [type=missing, input_value={'description': 'Read the...cretStr('**********')))}, input_type=dict]
    For further information visit https://

In [62]:
!ls -l

total 28
drwxr-xr-x 3 root root 4096 Aug 31 06:14 a29c26dd929742d0be97036227f5d929
drwxr-xr-x 3 root root 4096 Aug 31 05:52 cc5bf0b9e6d24ca49b70d2bf86901dd4
-rw-r--r-- 1 root root 1471 Aug 31 06:21 customer_feedback_analysis_report.txt
drwxr-xr-x 3 root root 4096 Aug 31 05:44 eb7f175d23d74370b3a1339a06b6d4cd
drwxr-xr-x 3 root root 4096 Aug 31 06:15 fdfea2ca8cee451282244fa13febff82
drwxr-xr-x 2 root root 4096 Aug 31 05:33 logs
drwxr-xr-x 1 root root 4096 Aug 29 13:22 sample_data


In [63]:
with open("customer_feedback_analysis_report.txt","r") as f:
    content = f.read()
    print(content)

Customer Feedback Analysis Report for Zuku

Introduction:
Zuku is a fiber internet service provider that has received mixed customer feedback from various online platforms, including social media, review sites, and forums.

Key Findings:
1. Poor customer service: Many customers have reported issues with their internet connection and difficulty in getting help from the customer service team.
2. Frequent internet disconnections: Some customers have reported frequent internet disconnections, which can be frustrating and affect their productivity.
3. Slow speeds: A few customers have reported slow internet speeds, which can be a major issue for those who rely on the internet for work or other important activities.

Recommendations:
1. Improve customer service: Zuku should focus on improving its customer service by providing timely and effective support to customers.
2. Address internet disconnections: Zuku should investigate the cause of frequent internet disconnections and take steps to p